In [ ]:
from astropy import units as u
from gammapy.datasets import Datasets
from gammapy.modeling.models import Models
from gammapy.modeling import Fit

import os
import sys
import importlib
path_my_modules = "/home/born-again/Documents/GitHub/my_modules"
module_path = os.path.abspath(f'{path_my_modules}/config')
if module_path not in sys.path:
    sys.path.append(module_path)

import cfg
importlib.reload(cfg)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_utilities}')
if module_path not in sys.path:
    sys.path.append(module_path)

import utilities as utl
importlib.reload(utl)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_plot_style}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import plotter
importlib.reload(plotter)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_spectral_models}')
if module_path not in sys.path:
    sys.path.append(module_path)

import spectral_models as spec
importlib.reload(spec)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_lhaaso_analysis}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import lhaaso_analysis as lhaaso
importlib.reload(lhaaso)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_gammapy_catalogs}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import gammapy_catalogs as gammapy_cat
importlib.reload(gammapy_cat)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_cta_simulation}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import cta_simulation as cta
importlib.reload(cta)

%matplotlib inline
import matplotlib.pyplot as plt # A collection of command style functions


path_my_plot_style = f"{path_my_modules}/{cfg.dir_plot_style}/my_plot_style.txt" 
plt.style.use(path_my_plot_style)

In [ ]:
path_CTA_datasets = f"/home/born-again/Documents/GitHub/JCAP-a/my_notebooks/pulsars/analysis/datasets/"
path_roi_datasets = f"/home/born-again/Documents/GitHub/JCAP-a/my_notebooks/counterparts_analysis/analysis/datasets/"

In [ ]:
roi_name = "LHAASO_J1825-1326_roi_1.0deg_e_ref_min_100.0GeV"
datasets_name = "HESS_J1825-137-CTA-PSR_J1826-1334_50h"
path_file = f"{path_CTA_datasets}/{roi_name}/{datasets_name}"
datasets_joint =Datasets.read(filename=f"{path_file}/datasets{cfg.format_yaml}", filename_models=f"{path_file}/models{cfg.format_yaml}")
models_joint = Models()
models_joint.extend(datasets_joint.models)
dict_leg_style = utl.load_dictionary(name=cfg.dict_leg_style, path_file=path_file)
dict_sep = utl.load_dictionary(name=cfg.dict_separation, path_file=path_file)

In [ ]:
print(datasets_joint)

In [ ]:


model =models_joint[-1]
models = [model]
datasets = Datasets([datasets_joint[7], datasets_joint[8]])

dict_leg_style = utl.load_dictionary(name=cfg.dict_leg_style, path_file=path_file)
dict_sep = utl.load_dictionary(name=cfg.dict_separation, path_file=path_file)

utl.load_dictionary(name=cfg.dict_separation, path_file=path_file)

dict_leg_style

dict_leg_style = plotter.set_leg_style(dict_leg_style, models = models)
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plotter.plot_SED(
    name = "model_name", 
    dict_plot_limits=dict_plot_limits,
    datasets=datasets, 
    models=models, 
    dict_leg_style=dict_leg_style, 
)

In [ ]:
dataset_CTA = datasets_joint[-1].copy()
sky_model_CTA_in = model.copy(name=f"{datasets_joint[-1].name} {model.spectral_model.tag[1]}")

In [ ]:
model = spec.sky_model_ecpl()
datasets.models = [model]
fit_joint = Fit(backend="sherpa")
result_joint = fit_joint.run(datasets=datasets)
print(result_joint)

In [ ]:
dict_leg_style = plotter.set_leg_style(dict_leg_style, models = [model])
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plotter.plot_SED(
    name = "model_name", 
    dict_plot_limits=dict_plot_limits,
    datasets=datasets, 
    models=[model, sky_model_CTA_in], 
    dict_leg_style=dict_leg_style, 
)

In [ ]:
list_data =[]
names = ["e_ref", "dnde", "dnde_errp", "dnde_errn", "is_ul"]
new_names = ["energy", "flux", "flux_error_hi", "flux_error_lo", "ul"]

In [ ]:
for index, dataset in enumerate(datasets):
    table = dataset.data.to_table(sed_type="dnde")
    table.rename_columns(names, new_names)
    list_data.append(table[new_names])

In [ ]:
list_data

In [ ]:
from astropy import units as u
import matplotlib.pyplot as plt
import naima
from gammapy.modeling.models import Models, NaimaSpectralModel, SkyModel
from naima.models import ExponentialCutoffPowerLaw, InverseCompton

In [ ]:
import astropy.units as u
import numpy as np

import naima

################################################################################
#
# This file shows a few example model functions (with associated priors, labels
# and p0 vector), that can be used as input for naima.run_sampler
#
################################################################################

#
# RADIATIVE MODELS
#
# Pion decay
# ==========

PionDecay_ECPL_p0 = np.array((46, 2.34, np.log10(80.0)))
PionDecay_ECPL_labels = ["log10(norm)", "index", "log10(cutoff)"]

# Prepare an energy array for saving the particle distribution
proton_energy = np.logspace(-3, 2, 50) * u.TeV


def PionDecay_ECPL(pars, data):
    amplitude = 10 ** pars[0] / u.TeV
    alpha = pars[1]
    e_cutoff = 10 ** pars[2] * u.TeV

    ECPL = naima.models.ExponentialCutoffPowerLaw(
        amplitude, 30 * u.TeV, alpha, e_cutoff
    )
    PP = naima.models.PionDecay(ECPL, nh=1.0 * u.cm ** -3)

    model = PP.flux(data, distance=1.0 * u.kpc)
    # Save a realization of the particle distribution to the metadata blob
    proton_dist = PP.particle_distribution(proton_energy)
    # Compute the total energy in protons above 1 TeV for this realization
    Wp = PP.compute_Wp(Epmin=1 * u.TeV)

    # Return the model, proton distribution and energy in protons to be stored
    # in metadata blobs
    return model, (proton_energy, proton_dist), Wp


def PionDecay_ECPL_lnprior(pars):
    logprob = naima.uniform_prior(pars[1], -1, 5)
    return logprob


# Inverse Compton with the energy in electrons as the normalization parameter
# ===========================================================================

IC_We_p0 = np.array((40, 3.0, np.log10(30)))
IC_We_labels = ["log10(We)", "index", "log10(cutoff)"]


def IC_We(pars, data):
    # Example of a model that is normalized though the total energy in electrons

    # Match parameters to ECPL properties, and give them the appropriate units
    We = 10 ** pars[0] * u.erg
    alpha = pars[1]
    e_cutoff = 10 ** pars[2] * u.TeV

    # Initialize instances of the particle distribution and radiative model
    # set a bogus normalization that will be changed in third line
    ECPL = naima.models.ExponentialCutoffPowerLaw(
        1 / u.eV, 10.0 * u.TeV, alpha, e_cutoff
    )
    IC = naima.models.InverseCompton(ECPL, seed_photon_fields=["CMB"])
    IC.set_We(We, Eemin=1 * u.TeV)

    # compute flux at the energies given in data['energy']
    model = IC.flux(data, distance=1.0 * u.kpc)

    # Save this realization of the particle distribution function
    elec_energy = np.logspace(11, 15, 100) * u.eV
    nelec = ECPL(elec_energy)

    return model, (elec_energy, nelec)


def IC_We_lnprior(pars):
    logprob = naima.uniform_prior(pars[1], -1, 5)
    return logprob

def ElectronIC(pars, data):
    """
    Define particle distribution model, radiative model, and return model flux
    at data energy values
    """

    ECPL = ExponentialCutoffPowerLaw(
        pars[0] / u.eV, 10.0 * u.TeV, pars[1], 10 ** pars[2] * u.TeV
    )
    IC = InverseCompton(ECPL, seed_photon_fields=["CMB"])

    return IC.flux(data, distance=1.0 * u.kpc)

#
# FUNCTIONAL MODELS
#
# Exponential cutoff powerlaw
# ===========================

ECPL_p0 = np.array((1e-12, 2.4, np.log10(15.0)))
ECPL_labels = ["norm", "index", "log10(cutoff)"]


def ECPL(pars, data):
    # Get the units of the flux data and match them in the model amplitude
    amplitude = pars[0] * data["flux"].unit
    alpha = pars[1]
    e_cutoff = (10 ** pars[2]) * u.TeV
    ECPL = naima.models.ExponentialCutoffPowerLaw(
        amplitude, 1 * u.TeV, alpha, e_cutoff
    )

    return ECPL(data)


def ECPL_lnprior(pars):
    logprob = naima.uniform_prior(pars[0], 0.0, np.inf) + naima.uniform_prior(
        pars[1], -1, 5
    )
    return logprob


# Log-Parabola or Curved Powerlaw
# ===============================

LP_p0 = np.array((1.5e-12, 2.7, 0.12))
LP_labels = ["norm", "alpha", "beta"]


def LP(pars, data):
    amplitude = pars[0] * data["flux"].unit
    alpha = pars[1]
    beta = pars[2]
    LP = naima.models.LogParabola(amplitude, 1 * u.TeV, alpha, beta)
    return LP(data)


def LP_lnprior(pars):
    logprob = naima.uniform_prior(pars[0], 0.0, np.inf) + naima.uniform_prior(
        pars[1], -1, 5
    )
    return logprob

def lnprior(pars):
    # Limit amplitude to positive domain
    logprob = naima.uniform_prior(pars[0], 0.0, np.inf)
    return logprob

In [ ]:
## Set initial parameters and labels
p0 = np.array((1e30, 3.0, np.log10(30)))
labels = ["norm", "index", "log10(cutoff)"]

## Run sampler
sampler, pos = naima.run_sampler(
    data_table=list_data,
    p0=PionDecay_ECPL_p0,
    labels=PionDecay_ECPL_labels,
    model=PionDecay_ECPL,
    prior=PionDecay_ECPL_lnprior,
    nwalkers=32,
    nburn=100,
    nrun=20,
    threads=4,
    prefit=True,
    interactive=True,
)


In [ ]:
 ## Save run results to HDF5 file (can be read later with naima.read_run)
naima.save_run("RXJ1713_IC_run.hdf5", sampler)

## Diagnostic plots with labels for the metadata blobs
naima.save_diagnostic_plots(
    "RXJ1713_IC",
    sampler,
    sed=True,
    last_step=False,
    blob_labels=[
        "Spectrum",
        "Electron energy distribution",
        "$W_e (E_e>1\, \mathrm{TeV})$",
    ],
)
naima.save_results_table("RXJ1713_IC", sampler)



In [ ]:
 naima.plot_fit(
     sampler, 
     modelidx=0, 
     label=None, 
     sed=True, 
     last_step=False, 
     n_samples=100, 
     confs=None, 
     ML_info=False, 
     figure=None, 
     plotdata=None, 
     plotresiduals=None, 
     e_unit=None, 
     e_range=[100*u.GeV, 100*u.TeV],
     e_npoints=100, 
     threads=None, 
     xlabel=None, 
     ylabel=None, 
     ulim_opts={}, 
     errorbar_opts={})

In [ ]:
import naima
from astropy.io import ascii

# Use plot_data from naima to plot the observed spectra
data = ascii.read("CrabNebula_spectrum.ecsv")
figure = naima.plot_data(data, e_unit=u.eV)
ax = figure.axes[0]

In [ ]:
data

In [ ]:
from datetime import datetime
import time
start_1 = time.perf_counter()

start_2 = datetime.now()
import warnings

#suppress warnings
warnings.filterwarnings('ignore')

import gammapy
from astropy import units as u
import numpy as np
from astropy.io import ascii
import collections
import sys, os
import matplotlib.pyplot as plt # A collection of command style functions
%matplotlib inline
from IPython.display import display

import math

import importlib

from gammapy.catalog import SourceCatalog3FHL
from gammapy.makers import SpectrumDatasetMaker, SafeMaskMaker, ReflectedRegionsBackgroundMaker
from gammapy.modeling import Fit
from gammapy.data import Observation, Observations, observatory_locations, DataStore, EventList
from gammapy.datasets import SpectrumDatasetOnOff, SpectrumDataset, Datasets
from gammapy.irf import load_cta_irfs, EffectiveAreaTable2D, load_irf_dict_from_file

from gammapy.maps import MapAxis, RegionGeom

from gammapy.modeling.models import (
    EBLAbsorptionNormSpectralModel,
    Models,
    PowerLawSpectralModel,
    SkyModel,
)

from gammapy.irf import EffectiveAreaTable2D

from numpy.random import RandomState

from scipy.stats import chi2, norm

from gammapy.estimators import FluxPointsEstimator
from gammapy.estimators import FluxPoints
from gammapy.datasets import FluxPointsDataset

from gammapy.modeling.models import (
    PowerLawSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    LogParabolaSpectralModel,
    SkyModel,
    Models, 
)

# astropy imports
from astropy.coordinates import SkyCoord, Angle

from astropy import units as u
from astropy.io import fits
from astropy.table import Table, Column

from astropy import cosmology
from astropy.cosmology import WMAP5, WMAP7
import astropy.cosmology.units as cu
from astropy.coordinates import Distance

from gammapy.estimators import SensitivityEstimator

# astropy affiliated packages imports
from regions import CircleSkyRegion

from gammapy.stats import WStatCountsStatistic
from gammapy.stats import CashCountsStatistic
from scipy.stats import sem
from gammapy.maps import Map
from regions import PointSkyRegion

from pathlib import Path

path_my_modules = "/home/born-again/Documents/GitHub/CTA_projects/my_modules"

plt.style.use(f"{path_my_modules}/plot_style/my_plot_style.txt")

import os
import sys
import importlib
module_path = os.path.abspath(f'{path_my_modules}/config')
if module_path not in sys.path:
    sys.path.append(module_path)

import cfg
importlib.reload(cfg)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_utilities}')
if module_path not in sys.path:
    sys.path.append(module_path)

import utilities as utl
importlib.reload(utl)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_spectral_models}')
if module_path not in sys.path:
    sys.path.append(module_path)

import spectral_models as spec
importlib.reload(spec)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_gammapy_catalogs}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import gammapy_catalogs as gammapy_cat
importlib.reload(gammapy_cat)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_lhaaso_analysis}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import lhaaso_analysis as lhaaso
importlib.reload(lhaaso)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_cta_simulation}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import cta_simulation as cta
importlib.reload(cta)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_plot_style}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import plotter
importlib.reload(plotter)

In [ ]:
roi_name = "LHAASO_J1825-1326_roi_1.0deg_e_ref_min_100.0GeV"
datase_name = "HESS_J1826-130-CTA-PSR_J1826-1256_50h"
path_datasets = f"/home/born-again/Documents/GitHub/JCAP_2023-1/my_notebooks/pulsars/analysis/datasets/{roi_name}"


In [ ]:
datasets_joint = utl.read_datasets_models(
    region_of_interest, 
    utl.name_to_txt(datasets_name), 
    f"{path_datasets}/{datasets_name}"
)
models_joint = Models()
models_joint.extend(datasets_joint.models)
path_file = f"{path_datasets}/{utl.name_to_txt(datasets_name)}"
dict_leg_style = utl.load_dictionary(region_of_interest, datasets_name, cfg.dict_leg_style, path_file)
dict_sep = utl.load_dictionary(region_of_interest, datasets_name, cfg.dict_separation, path_file)

In [ ]:
path_my_modules = "/home/born-again/Documents/GitHub/CTA_projects/my_modules"

module_path = os.path.abspath(f'{path_my_modules}/spectral_models')
if module_path not in sys.path:
    sys.path.append(module_path)

import spectral_models
importlib.reload(spectral_models)
from spectral_models import (
    sky_model_pl,
    sky_model_ecpl,
    sky_model_lp,
    sky_model_bpl,
)

module_path = os.path.abspath(f'{path_my_modules}/utilities')
if module_path not in sys.path:
    sys.path.append(module_path)

import utilities
importlib.reload(utilities)
from utilities import (
    mkdir_sub_directory, 
    write_tables_fits, 
    write_tables_csv, 
    load_catalogs_from_gammapy, 
    name_to_txt,
)

module_path = os.path.abspath(f'{path_my_modules}/config')
if module_path not in sys.path:
    sys.path.append(module_path)

import cfg
importlib.reload(cfg)

In [ ]:
module_path = os.path.abspath(f'{path_my_modules}/plot_style')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import plot
importlib.reload(plot)

In [ ]:
module_path = os.path.abspath(f'{path_my_modules}/lhaaso')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import lhaaso
importlib.reload(lhaaso)


In [ ]:
module_path = os.path.abspath(f'{path_my_modules}/hawc')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import hawc
importlib.reload(hawc)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # A collection of command style functions


path_my_plot_style = f"{path_my_modules}/plot_style/my_plot_style_2.txt" 
plt.style.use(path_my_plot_style)

In [ ]:
from gammapy.utils.check import check_tutorials_setup
from gammapy.visualization.utils import plot_contour_line
check_tutorials_setup()

In [ ]:
import pandas as pd 
def create_df_latex(sky_model, pulsar_name):
    
    table = sky_model.parameters.to_table()
    df = table.to_pandas()

    coluns_name = []
    df_column = []
    coluns_name.append("Pulsar name")
    df_column.append(pulsar_name)
    
    for i, name in enumerate(df["name"]):
        if i<=3:
            value = df["value"][i]
            error = df["error"][i]
            frozen = df["frozen"][i]
            unit = df["unit"][i]
            if frozen == True:
                value = "{:.0f}".format(value)
            else: 
                value = "{:.2e} ({:.2e})".format(value, error)

            if unit:
                name = f"{name} ({unit})"

            coluns_name.append(name)
            df_column.append(value)
    df = pd.DataFrame([df_column], columns = coluns_name)
    df_tex  = df.to_latex(index=False)
    return print(df_tex)

In [ ]:
from astropy.coordinates import SkyCoord, Distance

def set_pulsar_info(dict_pulsars, pulsar_index):
    """
    Sets the pulsar info into a dictionary
    """
    pulsar_name = list(dict_pulsars.keys())[pulsar_index]
    position_RA = list(dict_pulsars.values())[pulsar_index]["position"][0]
    position_dec = list(dict_pulsars.values())[pulsar_index]["position"][1]
    pulsar_pos = SkyCoord(position_RA, position_dec) # Source Position
    pulsar_dist = Distance(list(dict_pulsars.values())[pulsar_index]["distance"])
    pulsar_red = float(pulsar_dist.compute_z()) # The source redshift for this distance assuming its physical distance is a luminosity distance.

    return  {
        "name": pulsar_name,
        "position": pulsar_pos,
        "distance": pulsar_dist,
        "redshift": pulsar_red
    }

In [ ]:
def get_dict_pulsars():  
    dict_pulsars = {
        'PSR J1826-1334': {
            'position': (276.554896, -13.57967) * u.degree, 
            'distance': 3.1 * u.kpc,
            'age': 21.4 * u.kyr,
            'luminosity': 2.8e+36 * u.Unit("erg s-1")
        },
         'PSR J1826-1256': {
            'position': (276.53554, -12.94250) * u.degree, 
            'distance': 1.6 * u.kpc,
             'age': 14.4 * u.kyr,
             'luminosity': 3.6e+36 * u.Unit("erg s-1")
        },
         'PSR J1837-0604': {
            'position': (279.43146, -6.0803) * u.degree, 
            'distance': 4.8 * u.kpc,
            'age': 33.8 * u.kyr,
            'luminosity': 2.0e+36 * u.Unit("erg s-1")
        },
         'PSR J1838-0537': {
            'position': (279.73342, -5.6192) * u.degree, 
            'distance': 1.3 * u.kpc,
             'age': 4.9 * u.kyr,
             'luminosity': 6.0e+36 * u.Unit("erg s-1")
        },
    }
    return dict_pulsars

In [ ]:
def get_dataset_CTA(pulsar_index, irf_name, sky_model_name):
    dict_pulsars = get_dict_pulsars()
    pulsar_info = set_pulsar_info(dict_pulsars, pulsar_index)
    display(pulsar_info)
    file_name = utilities.create_file_name(pulsar_info, irf_name, sky_model_name)
    table_CTA = Table.read(f"{path_CTA_tables}/{file_name}{cfg.format_csv}",format='ascii', delimiter=' ', comment='#')
    datasets_names = f"CTA - {pulsar_info['name']} (5h)"
    # To read only models
    models = Models.read(f"{path_CTA_models}/{file_name}_model_out.yaml")
    sky_model_CTA = models[0].copy(name=f"{sky_model_name}_CTA", datasets_names = datasets_names)
    print(sky_model_CTA)
    return utilities.ds_fp_from_table_fp(table_CTA, sky_model_CTA, source_name=datasets_names), sky_model_CTA
    

In [ ]:
def plt_savefig(path_file, file_name):
    ''' Saves figures (.png and .pdf) in the path_child directoty    
    plt_savefig(path_child, child_name)
    >>> plt.savefig(file, bbox_inches='tight')
    '''
    formats_file = [".png", ".pdf"]
    for format_file in formats_file: 
        file = path_file / f'{file_name}{format_file}'
        plt.savefig(file, bbox_inches='tight')

In [ ]:
from astropy import units as u
def plot_SED(
    datasets = None,  
    models = None,
    dict_leg_style = None, 
    region_of_interest = None,
    sed_type = "e2dnde", 
    dict_plot_axis =  dict(
    label =  (r'$\rm{E\ [TeV] }$', r'$\rm{E^2\ J(E)\ [TeV\ cm^{-2}\ s^{-1}] }$'),
    units =  (          'TeV',                       'TeV  cm-2     s-1')
),
    dict_plot_limits = dict(
        energy_bounds = [1e-5, 3e2] * u.TeV,
        ylim = [1e-23, 1e-7]
    ),
    dict_leg_place = dict(
#         bbox_to_anchor = (0, -0.45), # Set legend outside plot
        ncol=3, 
        loc='lower left', 
    ),
    name = "SED" 
):    
    
    ax = plt.subplot()
    
    ax.xaxis.set_units(u.Unit(dict_plot_axis['units'][0]))
    ax.yaxis.set_units(u.Unit(dict_plot_axis['units'][1]))

    kwargs = {
        "ax": ax, 
        "sed_type": sed_type,
#         "uplims": True
    }

#     while len(cfg.markers) < len(datasets) + 1:
#          cfg.markers.extend( cfg.markers)
                        
    for index, dataset in enumerate(datasets):
        color = dict_leg_style[dataset.name][0]
        marker = dict_leg_style[dataset.name][1]
        

        label = dataset.name
        if dataset.name == "HESS J1826-130: gamma-cat":
            label = "HESS J1826-130 (2017)"
        if dataset.name == "HESS J1826-130: hgps":
            label = "HESS J1826-130 (2018)"
        if dataset.name == "HESS J1825-137: gamma-cat":
            label = "HESS J1825-137 (2006)"
        if dataset.name == "HESS J1825-137: hgps":
            label = "HESS J1825-137 (2018)"
            
            
        dataset.data.plot(
                    label = label, 
                    marker = marker, 
                    color=color,
                    **kwargs
                )
    if models:
        name_aux = ""
        for index, model in enumerate(models):
            linestyle = dict_leg_style[model.name][1]
            color = dict_leg_style[model.name][0]
            spectral_model = model.spectral_model
            
#             spectral_model.plot(label = f"{model.name} (fit)", energy_bounds=dict_plot_limits['energy_bounds'],   marker = ',', color="black", **kwargs)
            energy_bounds = [7e-2, 8e2] * u.TeV
#             energy_bounds=dict_plot_limits['energy_bounds']
            spectral_model.plot(energy_bounds=energy_bounds,  linestyle = linestyle, marker = ',', color=color, **kwargs)

            spectral_model.plot_error(energy_bounds=energy_bounds,**kwargs)
    else:
        name_aux = "flux_points_"
    ax.set_ylim(dict_plot_limits['ylim'])
    ax.set_xlim(dict_plot_limits['energy_bounds'])
    
    ax.legend(**dict_leg_place)
    
    plt.xlabel(dict_plot_axis['label'][0])   
    plt.ylabel(dict_plot_axis['label'][1])
    
    file_name = f"{name_aux}{name_to_txt(name)}"
        
    path_file =  utilities.get_path_SED(region_of_interest)  
    plt_savefig(path_file, file_name)
    
    plt.savefig(path_file, bbox_inches='tight')
#     plt.grid(which="both")
    plt.show()
    
    return

In [ ]:
from gammapy.modeling import Fit
from gammapy.datasets import Datasets
from gammapy.modeling.models import Models

def fit_Datasets(datasets, sky_model, add_name = None):
    if add_name is not None:
        model_name = f"{name_to_txt(sky_model.name)}_{add_name}"
    else:
        model_name = name_to_txt(sky_model.name)
        
    datasets = Datasets(datasets)
    sky_model = sky_model.copy(name = model_name)
        
    datasets.models = sky_model
    # print(datasets)
    fitter = Fit()
    result_fit = fitter.run(datasets=datasets)
    print(result_fit.parameters.to_table())
    print(result_fit.total_stat)
    models = Models(sky_model.copy(name= model_name)) 
    file_path = utilities.get_path_models(region_of_interest)
    # To save only the models
    models.write(f"{file_path}/{model_name}.yaml", overwrite=True)
    return sky_model


In [ ]:
# from gammapy.datasets import FluxPointsDataset
from astropy.coordinates import SkyCoord
from astropy import units as u

import pickle

def create_catalogs_region_of_interest(region_of_interest):
    """
    Gets catalogs subset (only sources within the radius of the region of interest)
    """
    
    source_catalogs = utilities.load_catalogs_from_gammapy()
    
    source_position = region_of_interest["position"] 
    radius_roi = region_of_interest["radius_roi"] 
        
    catalogs_roi = []
    catalogs_no_counterparts = []
    numbers_catalogs_roi = 0
    
    for catalog in source_catalogs:        
        # Selects only sources within the region of interest. 
        mask_roi = source_position.separation(catalog.positions) < radius_roi 
        
        if len(catalog[mask_roi].table):
            catalogs_roi.append(catalog[mask_roi])
            numbers_catalogs_roi += 1
        else:
            catalogs_no_counterparts.append(f"{catalog.tag}: {catalog.description}")
    
            
    if numbers_catalogs_roi:
        utilities.pickling_catalog_roi(catalogs_roi, region_of_interest)
        print(f"\n{numbers_catalogs_roi} catalogs with sources within the region of interest:", end = "\n\n")
        for catalog in catalogs_roi:
            print(f"{catalog.tag}: {catalog.description}")
            display(catalog.table)
    else:
        print("No catalogs with sources in the region of interest!", end = "\n\n")

    if numbers_catalogs_roi and len(catalogs_no_counterparts):
        print("Catalogs without sources within the region of interest:", end = "\n\n")
        for index, catalog_no_counterpart in enumerate(catalogs_no_counterparts):                            
            print(catalog_no_counterpart)

    return catalogs_roi

In [ ]:
from gammapy.datasets import FluxPointsDataset
from astropy.coordinates import SkyCoord
from astropy import units as u
from gammapy.modeling.models import SkyModel, Models
from gammapy.datasets import Datasets

def get_datasets_flux_points(region_of_interest):
    '''
    Select a catalog subset (only sources within a region of interest)
    '''
    
    # Creates the directories to save the flux points tables 
#     directory_file = f'{cfg.dir_flux_points_tables}/{region_of_interest["roi_name"]}'
#     path_analysis, path_tables = mkdir_sub_directory(cfg.dir_analysis, directory_file)
    path_tables = utilities.get_path_tables(region_of_interest)
    path_tables, path_file = mkdir_sub_directory(str(path_tables), "counterparts_gammapy")

    path_datasets = utilities.get_path_datasets(region_of_interest)
    try:
        catalogs_roi = utilities.unpickling_catalog_roi(region_of_interest)
    except:
        catalogs_roi = create_catalogs_region_of_interest(region_of_interest)
        
    datasets_counterparts = Datasets() # global datasets object
    models_counterparts = Models()  # global models object
    counterparts = [] # global sources object
    
    n_counterparts = 0 # number of counterparts
    n_flux_points = 0 # number of flux points tables
    
    e_ref_min = region_of_interest["e_ref_min"] 
    e_ref_max = region_of_interest["e_ref_max"]
    
    #############################
    if region_of_interest["name"] == 'LHAASO J1839-0545':
        datasets_counterparts.append(dataset_HESSJ1837_gammacat)
        n_counterparts+=1 
        datasets_counterparts.append(dataset_HESSJ1837_hgps)
        n_counterparts+=1
    #############################
        
    for catalog in catalogs_roi:
        cat_tag = catalog.tag
        for counterpart in catalog:
            n_counterparts+=1   
            counterpart_name = counterpart.name            
            try:
                flux_points = counterpart.flux_points

                counterpart_spectral_model = counterpart.spectral_model()
                spectral_model_tag = counterpart_spectral_model.tag[0]
                spectral_model_tag_short = counterpart_spectral_model.tag[1]
                
                if cat_tag != 'gamma-cat' and cat_tag != 'hgps':
                    ds_name = f"{counterpart_name}"
                else:
                    ds_name = f"{counterpart_name}: {cat_tag}"
                     
                file_name = name_to_txt(ds_name)
                
                
                counterpart_model = SkyModel(
                    name = f"{file_name}_{counterpart_spectral_model.tag[1]}",
                    spectral_model = counterpart_spectral_model,
                    datasets_names=ds_name
                )
        
                ds = FluxPointsDataset(
                    models = counterpart_model,
                    data = flux_points, 
                    name =  ds_name   
                )
                
                if any([e_ref_min !=  None, e_ref_max !=  None]):
                    ds = cut_flux_points_energy_range(ds, e_ref_min, e_ref_max)
                
                n_flux_points+=1
                models_counterparts.append(counterpart_model)  # Add the counterpart_model to models()
                table = ds.data.to_table(sed_type = cfg.sed_type_dnde, formatted = True)
#                 print(ds_name)
#                 print(flux_points.to_table(formatted = True))
                
                counterparts.append(counterpart)
                datasets_counterparts.append(ds)
                
                

                # Writes the flux points table in the csv/fits format
                write_tables_csv(table, path_file, file_name)
                write_tables_fits(table, path_file, file_name)
                
            except Exception as error:
                # By this way we can know about the type of error occurring
                print(f'The error is: ({counterpart_name}) {error}') 
            
    datasets_counterparts.models = models_counterparts
    # To save datasets and models
    path_datasets, path_file = mkdir_sub_directory(str(path_datasets), "counterparts_gammapy")
    datasets_counterparts.write(filename=f"{path_file}/datasets{cfg.format_yaml}", filename_models=f"{path_file}/models{cfg.format_yaml}", overwrite=True
    )
            
    print(f"Total number of counterparts: {n_counterparts}")
    print(f"Total number of flux points tables: {n_flux_points}")
    return counterparts, datasets_counterparts, models_counterparts

In [ ]:
def get_datasets_flux_points_outside_gammapy(region_of_interest):
    '''
    Select a catalog subset (only sources within a region of interest)
    '''
    path_tables = utilities.get_path_tables(region_of_interest)
    path_datasets = utilities.get_path_datasets(region_of_interest)
    
    datasets_counterparts = Datasets() # global datasets object
    models_counterparts = Models()  # global models object
    
    n_counterparts = 0 # number of counterparts
    n_flux_points = 0 # number of flux points tables
    
    e_ref_min = region_of_interest["e_ref_min"] 
    e_ref_max = region_of_interest["e_ref_max"]
    
    roi_pos = region_of_interest["position"]
    radius_roi = region_of_interest["radius_roi"]

    
    dict_HAWC = hawc.get_dict_HAWC()
    dict_LHAASO = lhaaso.get_dict_LHAASO()
    for index, source_name in enumerate(list(dict_HAWC.keys())):
        source_pos = dict_HAWC[source_name]["position"]
        if roi_pos.separation(source_pos) <= radius_roi:
            ds = get_HAWC_dataset(source_name)
            if any([e_ref_min !=  None, e_ref_max !=  None]):
                    ds = cut_flux_points_energy_range(ds, e_ref_min, e_ref_max)
            counterpart_model = ds.models[0]
            models_counterparts.append(counterpart_model)
            datasets_counterparts.append(ds)
            
            table = ds.data.to_table(sed_type = cfg.sed_type_e2dnde, formatted = True)
             # Writes the flux points table in the csv/fits format
            file_name = source_name
            path_tables, path_file = mkdir_sub_directory(str(path_tables), "counterparts_outside_gammapy")
            write_tables_csv(table, path_file, file_name)
            write_tables_fits(table, path_file, file_name)

            n_flux_points+=1
            n_counterparts+=1 
            
    for index, source_name in enumerate(list(dict_LHAASO.keys())):
        source_pos = dict_LHAASO[source_name]["position"]
        if roi_pos.separation(source_pos) <= radius_roi:
            ds = get_LHAASO_dataset(source_name)
            if any([e_ref_min !=  None, e_ref_max !=  None]):
                    ds = cut_flux_points_energy_range(ds, e_ref_min, e_ref_max)
            counterpart_model = ds.models[0]
            models_counterparts.append(counterpart_model)
            datasets_counterparts.append(ds)
            
            table = ds.data.to_table(sed_type = cfg.sed_type_e2dnde, formatted = True)
             # Writes the flux points table in the csv/fits format
            file_name = source_name
            path_tables, path_file = mkdir_sub_directory(str(path_tables), "counterparts_outside_gammapy")
            write_tables_csv(table, path_file, file_name)
            write_tables_fits(table, path_file, file_name)

            n_flux_points+=1
            n_counterparts+=1
            
    if datasets_counterparts:
        datasets_counterparts.models = models_counterparts
        # To save datasets and models
        path_datasets, path_file = mkdir_sub_directory(str(path_datasets), "counterparts_outside_gammapy")
        datasets_counterparts.write(filename=f"{path_file}/datasets{cfg.format_yaml}", filename_models=f"{path_file}/models{cfg.format_yaml}", overwrite=True
        )
            
        print(f"Total number of counterparts: {n_counterparts}")
        print(f"Total number of flux points tables: {n_flux_points}")
    
        return datasets_counterparts, models_counterparts
    else: print("No counterparts in the ROI")

In [ ]:
import numpy as np
from gammapy.estimators import FluxPoints

def cut_flux_points_energy_range(dataset, e_ref_min, e_ref_max):
    
    if all([e_ref_min ==  None, e_ref_max ==  None]):
            raise Exception(f"Sorry, there is a error: e_ref_min is None and e_ref_max is None)") 
    
    flux_points = dataset.data
    models = dataset.models[0]      
    ds_name = dataset.name
    
    if e_ref_min != None:
        mask_energy = np.zeros(len(flux_points.to_table()), dtype=bool)

        for m, e_ref in enumerate(flux_points.energy_ref):
            if e_ref >= e_ref_min:
                mask_energy[m] = True

        flux_points_mask = flux_points.to_table()[mask_energy]
        flux_points = FluxPoints.from_table(flux_points_mask)
    
    if e_ref_max != None:
        mask_energy = np.zeros(len(flux_points.to_table()), dtype=bool)

        for m, e_ref in enumerate(flux_points.energy_ref):
            if e_ref <= e_ref_max:
                mask_energy[m] = True

        flux_points_mask = flux_points.to_table()[mask_energy]
        flux_points = FluxPoints.from_table(flux_points_mask)     
        
    return FluxPointsDataset(models = models, data = flux_points, name = ds_name)

In [ ]:
def set_leg_style(dict_leg_style, datasets = None, models = None, color = None, marker = None, linestyle = None):
    if all([datasets ==  None, models ==  None]):
        return print("Sorry, there is error: 'datasets =  None' and 'models =  None'")
    else: 
#             marker_ds = marker
#             color_ds = color
    
        if datasets !=  None:
            dict_leg_style = set_leg_style_datasets(dict_leg_style, datasets, color, marker)
        
        if models !=  None:
            dict_leg_style = set_leg_style_models(dict_leg_style, models, color, linestyle)
        
        return dict_leg_style

In [ ]:
def set_leg_style_datasets(dict_leg_style, datasets, color = None, marker = None):
    datasets = Datasets(datasets)
    marker_ds = marker
    color_ds = color
    if not marker_ds:
        while len(cfg.markers) < len(datasets) +1:
            cfg.markers.extend(cfg.markers)
    if not color_ds:      
        while len(colors) < len(datasets) +1:
            colors.extend(colors)

    for index, dataset in enumerate(datasets):
        if not color_ds:
            color = colors[index]

        if not color_ds:
            marker = cfg.markers[index]
        
        #############################
        if dataset.name.find('LHAASO') != -1:
            color = "red"
            marker = "o"
            
        if dataset.name.find('CTA') != -1:
            color = "blue"
            marker = "s"
        #############################    
        dict_leg_style[dataset.name] = (color, marker)
    return dict_leg_style

In [ ]:
def set_leg_style_models(dict_leg_style, models, color = None, linestyle = None):
    linestyles = ['solid','dotted','dashed','dashdot']
    models = Models(models)
    color_m = color
    linestyle_m = linestyle
    
    if not linestyle:
        while len(linestyles) < len(models) +1:
            linestyles.extend(linestyles)
    if not color_m:      
        while len(colors) < len(models) +1:
            colors.extend(colors)

    for index, model in enumerate(models):
        if not color_m:
            color = "black"
            
        linestyle = linestyles[index]
        dict_leg_style[model.name] = (color, linestyle)
    return dict_leg_style

In [ ]:
path_CTA_models = f"/home/born-again/Documents/GitHub/CTA_projects/my_notebooks/pulsars/analysis/models"
path_CTA_tables = "/home/born-again//Documents/GitHub/CTA_projects/my_notebooks/pulsars/analysis/tables"

### Data Selection

In [ ]:
# In this simulation, we use the CTA-1DC irfs shipped with gammapy
base_name = '/home/born-again/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/tutorials/data/caldb/data/cta/prod3b-v2/bcf'

irf_z = [20,40,60]
irf_h = [0.5, 5, 50]
irf_loc = [("cta_north", "North"),("cta_south", "South")]

In [ ]:
dict_pulsars = get_dict_pulsars()

In [ ]:
irf_zenith = 0
irf_hours = 1 
irf_site = 1 

irf_name = f'{irf_loc[irf_site][irf_site]}_z{irf_z[irf_zenith]}_{irf_h[irf_hours]}h'
irf_name

<a id='intro'></a>
🔝 [Back to Top](#indice)<br>
## 1. Introduction 
A Python code to search for possible γ-ray counterparts to the target source and to perform the spectral model fitting. This code selects the sources (in the Gammapy source catalogs) within the region of interest (centered in the position of the target source) and finds the best fit for the given spectrum model. 

### LHAASO/HAWC Datasets

In [ ]:
path_fp_HAWC = "/home/born-again/Documents/GitHub/CTA_projects/flux_points_outside_gammapy_catalogs/HAWC"
from gammapy.modeling.models import Models
from gammapy.datasets import Datasets
from gammapy.modeling.models import PowerLawSpectralModel
def get_HAWC_dataset(source_name):
    # sky model HAWC J1825-134
    # https://arxiv.org/pdf/2012.15275.pdf
    if source_name == "HAWC J1825-134":
        sky_model = sky_model_pl(
            index=2.28,
            amplitude="4.2e-15 TeV-1 cm-2 s-1",
            reference=18 * u.TeV,
        datasets_names = source_name
        )

    # sky model HAWC J1825-138
    # https://arxiv.org/pdf/2012.15275.pdf
    if source_name == "HAWC J1825-138":
        sky_model = sky_model_ecpl(
            amplitude=2.7e-14 * u.Unit("cm-2 s-1 TeV-1"),
            index=2.02,
            lambda_=1/27 * u.Unit("TeV-1"),
            reference=18 * u.TeV,
        datasets_names = source_name
        )

    # sky model HAWC J1826-128
    # https://arxiv.org/pdf/2012.15275.pdf
    if source_name == "HAWC J1826-128":
        sky_model = sky_model_ecpl(
            amplitude=2.7e-14 * u.Unit("cm-2 s-1 TeV-1"),
            index=1.2,
            lambda_=(1/24) * u.Unit("TeV-1"),
            reference=18 * u.TeV,
        datasets_names = source_name
        )

    # sky model eHWC J1825-134
    # https://arxiv.org/pdf/1909.08609.pdf
    if source_name == "eHWC J1825-134":
        sky_model = sky_model_ecpl(
            amplitude=2.12e-13 * u.Unit("cm-2 s-1 TeV-1"),
            index=2.12,
            lambda_= (1/61) * u.Unit("TeV-1"),
            reference=10 * u.TeV,
        datasets_names = source_name
        )

    # sky model eHWC J1907+063
    # https://arxiv.org/pdf/1909.08609.pdf
    if source_name == "eHWC J1907+063":
        sky_model = sky_model_lp(
             alpha=2.46,
            amplitude="0.95e-13 cm-2 s-1 TeV-1",
            reference=10 * u.TeV,
            beta=0.11,
        datasets_names = source_name
        )

    # sky model eHWC J2019+368
    # https://arxiv.org/pdf/1909.08609.pdf
    if source_name == "eHWC J2019+368":
        sky_model = sky_model_lp(
            alpha=2.08,
            amplitude="0.45e-13 cm-2 s-1 TeV-1",
            reference=10 * u.TeV,
            beta=0.26,
        datasets_names = source_name
        )

    table = hawc.HAWC_table_to_SED_format(path_fp_HAWC, name_to_txt(source_name))  
    return utilities.ds_fp_from_table_fp(table = table, sky_model = sky_model, source_name=source_name)

In [ ]:
path_fp_LHAASO = "/home/born-again/Documents/GitHub/CTA_projects/flux_points_outside_gammapy_catalogs/LHASSO_publishNature"

from gammapy.modeling.models import SkyModel, Models
from gammapy.modeling.models import LogParabolaSpectralModel
from pathlib import Path
from gammapy.datasets import Datasets
    
def get_LHAASO_dataset(source_name): 
    
    # sky model LHAASO J1825-1326
    if source_name == 'LHAASO J1825-1326':
        sky_model = sky_model_lp(
            alpha = 0.92,
            amplitude = "1e-12 cm-2 s-1 TeV-1",
            reference = 10 * u.TeV,
            beta = 1.19,
            datasets_names = source_name
        )
        
    # sky model LHAASO J1908+0621
    elif source_name == "LHAASO J1908+0621":
        sky_model = sky_model_lp(
        alpha = 2.27,
        amplitude = "1e-12 cm-2 s-1 TeV-1",
        reference = 10 * u.TeV,
        beta = 0.46,
        datasets_names = source_name
    )

    # sky model LHAASO J2226+6057
    elif source_name == "LHAASO J2226+6057":
        sky_model = sky_model_lp(
        alpha = 1.56,
        amplitude = "1e-12 cm-2 s-1 TeV-1",
        reference = 10 * u.TeV,
        beta = 0.88,
        datasets_names = source_name
    )
    else:
        sky_model = sky_model_pl(
            datasets_names = source_name
        )
     
    table = lhaaso.LHAASO_table_to_SED_format(path_fp_LHAASO, name_to_txt(source_name))
        
    return utilities.ds_fp_from_table_fp(table = table, sky_model = sky_model, source_name = source_name)

### HAWC Datasets

In [ ]:
models_HAWC = models_roi.select(name_substring="WC")
print(models_HAWC)

## Counterparts Analysis

In [ ]:
dict_LHAASO = lhaaso.get_dict_LHAASO()

In [ ]:
source_index = 1
source_name = list(dict_LHAASO.keys())[source_index]
source_RA = list(dict_LHAASO.values())[source_index]["position"].ra
source_dec = list(dict_LHAASO.values())[source_index]["position"].dec

In [ ]:
source_info = utilities.set_source_info(source_name, source_RA, source_dec)
# source_info

In [ ]:
radius_roi = 1 * u.Unit("deg")  # maximum angle of separation (in degrees)
e_ref_min = 100 * u.Unit("GeV")
# e_ref_max = 10 * u.Unit("TeV")

In [ ]:
# REVER: chamar (com source_name, source_RA, source_dec) source_info dentro
region_of_interest = utilities.create_region_of_interest(source_info, radius_roi, e_ref_min)
region_of_interest

In [ ]:
# df = utilities.create_data_frame_counterparts(region_of_interest)
# display(df)
# print(df.to_latex()) # Render object to a LaTeX tabular, longtable, or nested table.

In [ ]:
# print(df[["Source name","RA(deg)","dec.(deg)","Sep.(deg)","Flux points"]].to_latex(index=False)) # Render object to a LaTeX tabular, longtable, or nested table.
# print(df.to_latex(index=False)) # Render object to a LaTeX tabular, longtable, or nested table.

In [ ]:
sources_gammapy, datasets_gammapy, models_gammapy = get_datasets_flux_points(region_of_interest)

In [ ]:
datasets_outside_gammapy, models_outside_gammapy = get_datasets_flux_points_outside_gammapy(region_of_interest)

In [ ]:
datasets_roi = Datasets()
models_roi = Models()
for index, (dataset, model) in enumerate(zip(datasets_gammapy, models_gammapy)):
    datasets_roi.append(dataset)
    models_roi.append(model)
    print(index, dataset.name)
for index_, (dataset, model) in enumerate(zip(datasets_outside_gammapy, models_outside_gammapy)):
    datasets_roi.append(dataset)
    models_roi.append(model)
    print(index_+index+1, dataset.name)

In [ ]:
# To read datasets with models
path_datasets = utilities.get_path_datasets()
datasets_read = Datasets.read(f"{path_datasets}/datasets_{region_of_interest['roi_name']}.yaml", filename_models=f"{path_datasets}/models_{region_of_interest['roi_name']}.yaml")
for index, dataset in enumerate(datasets_read):
#     dataset.data.to_table(sed_type = cfg.sed_type_e2dnde, formatted = True)
#     dict_leg_style[datasets_read[index].name] = ("black", ">")
    print(index, dataset.name)

In [ ]:
datasets_roi[17].data.to_table()

In [ ]:
colors = ["aqua",
"fuchsia",
"peru",
"brown",
"chartreuse",
"chocolate",
"coral",
"crimson",
"darkblue",
# "goldenrod",
"cadetblue",
"pink",
"indigo",
"seagreen",
"khaki",
"darkmagenta",
          "orange",
"springgreen",
"lime",
"magenta",
"maroon",
"navy",
"olive",
"skyblue"          
"orange",
"orangered",
"orchid",
"pink",
"plum",
"purple",
"red",
"salmon",
"sienna",
"silver",
"tan",
"teal",
"azure",
"beige",
"ivory",
"black",
"tomato",
"turquoise",
"violet",
"aquamarine",
"wheat",
"white",
"cyan",
"blue",
"lightblue",
"yellowgreen"]

In [ ]:
dict_leg_style = set_leg_style({}, datasets = datasets_roi, color = None, marker = None)

In [ ]:
sed_type = cfg.sed_type_e2dnde
dict_plot_axis = dict(
    label =  (r'$\rm{E\ [TeV] }$', r'$\rm{E^2\ J(E)\ [TeV\ cm^{-2}\ s^{-1}] }$'),
    units =  (          'TeV',                       'TeV  cm-2     s-1')
)
dict_leg_place = dict(
    bbox_to_anchor = (0, -0.45), # Set legend outside plot
    ncol=3, 
    loc='lower left', 
)
dict_plot_limits = dict(
    energy_bounds = [1e-5, 3e3] * u.TeV,
    ylim = [1e-26, 1e-7]
)

In [ ]:
for dataset in datasets_roi:
    print(f"{dataset.name}: {dict_leg_style[dataset.name]}")

In [ ]:
path_my_plot_style = f"{path_my_modules}/plot_style/my_plot_style_2.txt" 
plt.style.use(path_my_plot_style)

datasets = Datasets(datasets_roi)
dict_plot_limits = dict(
    energy_bounds = [2e-2, 2e3] * u.TeV,
    ylim = [1e-16, 1e-9]
)

plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name="datasets_roi")

In [ ]:
plot.SED_from_catalogs(
    sources_gammapy, 
    datasets_gammapy, 
    models_gammapy,
    dict_leg_style,
    region_of_interest,
    sed_type = sed_type, 
    dict_plot_axis=dict_plot_axis,
    energy_bounds = [1e-5, 1e2] * u.TeV, 
    ylim = [1e-13, 1e-9]
)

### HESS J1825-137
[tevcat](http://tevcat.uchicago.edu/?mode=1&showsrc=115)

In [ ]:
# models = models_counterparts.select(datasets_names=cta_dataset.name, name_substring="1748")
models_HESS = models_roi.select(name_substring="HESS")
print(models_HESS)

In [ ]:
datasets = Datasets([datasets_roi[1],datasets_roi[2],datasets_roi[6],datasets_roi[9],datasets_roi[11],datasets_roi[13],datasets_roi[16],datasets_roi[17]])
datasets_HESSJ1825 = datasets.copy()

In [ ]:
models = models_HESS.select(name_substring="J1825-137")
models_in = models.copy()

In [ ]:
datasets = Datasets(datasets)
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest)

### Naima test

In [ ]:
path_file = utilities.get_path_tables()

In [ ]:
for index, dataset in enumerate(datasets):
    table = dataset.data.to_table(sed_type="e2dnde")
    display(table)
    write_tables_csv(table, path_file, f"naima_{index}")

#### gamma-cat

In [ ]:
model_index = 0

In [ ]:
#REVER name, datasets_names sky_model_ecpl,...
datasets_names = models[model_index].datasets_names
sky_model_in = sky_model_ecpl(datasets_names=datasets_names)
sky_model = sky_model_in.copy(name=sky_model_in.name,datasets_names = sky_model_in.datasets_names)
# sky_model = fit_Datasets(datasets,sky_model, add_name="fit")
sky_model = fit_Datasets(datasets,sky_model)

In [ ]:
# model = models_3fhl[0]
# print(model)

# # To freeze a single parameter
# model.spectral_model.index.frozen = True
# print(model)  # index is now frozen

# # To unfreeze a parameter
# model.spectral_model.index.frozen = False

# # To freeze all parameters of a model
# model.freeze()
# print(model)

# # To unfreeze all parameters (except parameters which must remain frozen)
# model.unfreeze()
# print(model)

In [ ]:
for model in (models):
    print(model.name)

In [ ]:
dict_leg_style = set_leg_style(dict_leg_style, models = models)

In [ ]:
datasets = Datasets(datasets)
models=[sky_model]
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets,models=models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name = "datasets_HESSJ1825")

In [ ]:
pulsar_index = 0
pulsar_name = "PSR J1826-1334"

In [ ]:
dataset_CTA, sky_model_CTA_in = get_dataset_CTA(pulsar_index, irf_name, sky_model.name)
sky_model_CTA = sky_model_CTA_in.copy(name=sky_model_CTA_in.name,datasets_names = sky_model_CTA_in.datasets_names)
dict_leg_style = set_leg_style(dict_leg_style, dataset_CTA, color="blue", marker="o")
datasets.append(dataset_CTA)

In [ ]:
sky_model_CTA= fit_Datasets(datasets, sky_model_CTA)
sky_model= fit_Datasets(datasets,sky_model)

In [ ]:
datasets = Datasets(datasets)
# models=[sky_model_CTA, sky_model]
models=[sky_model_CTA]
dict_leg_style = set_leg_style(dict_leg_style, models = models)
dict_plot_limits = dict(
    energy_bounds = [5e-2, 1e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, models = models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name= sky_model.name)

In [ ]:
from astropy import units as u
import matplotlib.pyplot as plt
import naima
from gammapy.modeling.models import Models, NaimaSpectralModel, SkyModel

particle_distribution = naima.models.ExponentialCutoffPowerLaw(
    1e30 / u.eV, 10 * u.TeV, 3.0, 30 * u.TeV
)
# radiative_model = naima.radiative.InverseCompton(
#     particle_distribution,
#     seed_photon_fields=["CMB", ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm**3]],
#     Eemin=100 * u.GeV,
# )

radiative_model = naima.models.PionDecay(particle_distribution)

model = NaimaSpectralModel(radiative_model, distance=1.5 * u.kpc)

# opts = {
#     "energy_bounds": [10 * u.GeV, 80 * u.TeV],
#     "sed_type": "e2dnde",
# }

# # Plot the total inverse Compton emission
# model.plot(label="IC (total)", **opts)

# # Plot the separate contributions from each seed photon field
# for seed, ls in zip(["CMB", "FIR"], ["-", "--"]):
#     model = NaimaSpectralModel(radiative_model, seed=seed, distance=1.5 * u.kpc)
#     model.plot(label=f"IC ({seed})", ls=ls, color="gray", **opts)

# plt.legend(loc="best")
# plt.grid(which="both")


In [ ]:
print(model)

In [ ]:
sky_model_naima = SkyModel(
    spectral_model=model,
    name="naima")

In [ ]:
sky_model_naima= fit_Datasets(datasets, sky_model_naima)

In [ ]:
dict_leg_style = set_leg_style(dict_leg_style, models=[sky_model_naima])

In [ ]:
datasets = Datasets(datasets)
# models=[sky_model_CTA, sky_model]
models=[sky_model_CTA,sky_model_naima]
dict_leg_style = set_leg_style(dict_leg_style, models=models)
dict_plot_limits = dict(
    energy_bounds = [5e-2, 1e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, models = models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name= sky_model.name)

In [ ]:
df_latex = create_df_latex(sky_model_CTA, pulsar_name)

In [ ]:
# datasets_HESS_J1825-137_gamma-cat_ecpl_CTA.yaml
# models_HESS_J1825-137_gamma-cat_ecpl_CTA.yaml
# To save datasets and models
datasets.models = [sky_model_CTA]
datasets.write(
    filename=f"datasets_{sky_model_CTA.name}.yaml", filename_models=f"models_{sky_model_CTA.name}.yaml", overwrite=True
)
# # To read only models
# models = Models.read("3fhl_models.yaml")
# print(models)
# To read datasets with models
# datasets_read = Datasets.read(f"datasets_{sky_model_CTA.name}.yaml", filename_models=f"models_{sky_model_CTA.name}.yaml")
# print(datasets_read)

In [ ]:
datasets_HESSJ1825_CTA = Datasets.read(f"datasets_{sky_model_CTA.name}.yaml", filename_models=f"models_{sky_model_CTA.name}.yaml")
print(datasets_HESSJ1825_CTA)

In [ ]:
datasets = Datasets(datasets_HESSJ1825_CTA)
# models=[sky_model_CTA, sky_model]
models=[datasets_HESSJ1825_CTA.models[0]]

dict_plot_limits = dict(
    energy_bounds = [5e-2, 1e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, models = models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name= sky_model.name)

### HESS J1826-130
[tevcat](http://tevcat.uchicago.edu/?mode=1;id=271)

In [ ]:
datasets = Datasets([datasets_roi[0], datasets_roi[3], datasets_roi[4], datasets_roi[5],datasets_roi[7],datasets_roi[8],datasets_roi[10]
,datasets_roi[12],datasets_roi[17]])
datasets_HESSJ1826 = datasets.copy()

In [ ]:
models = models_HESS.select(name_substring="J1826-130")
models_in = models.copy()

In [ ]:
datasets = Datasets(datasets)
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest)

In [ ]:
model_index = 0

In [ ]:
#REVER name, datasets_names sky_model_ecpl,...
datasets_names = models[model_index].datasets_names
sky_model_in = sky_model_ecpl(datasets_names=datasets_names)
sky_model = sky_model_in.copy(name=sky_model_in.name,datasets_names = sky_model_in.datasets_names)
sky_model = fit_Datasets(datasets,sky_model)

In [ ]:
datasets = Datasets(datasets)
models=[sky_model]
dict_leg_style = set_leg_style(dict_leg_style, models = models)

dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets,models=models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name = "datasets_HESSJ1825")

In [ ]:
dict_pulsars

In [ ]:
pulsar_index = 1
pulsar_name = "PSR J1826-1256"

In [ ]:
dataset_CTA, sky_model_CTA_in = get_dataset_CTA(pulsar_index, irf_name, sky_model.name)
sky_model_CTA = sky_model_CTA.copy(name=sky_model_CTA_in.name,datasets_names = sky_model_CTA_in.datasets_names)
dict_leg_style = set_leg_style(dict_leg_style, dataset_CTA, color="blue", marker="o")
datasets.append(dataset_CTA)

In [ ]:
sky_model_CTA= fit_Datasets(datasets, sky_model_CTA)
sky_model= fit_Datasets(datasets,sky_model)

In [ ]:
df_tex = create_df_latex(sky_model_CTA, pulsar_name)

In [ ]:
print(df_tex)

In [ ]:
df.to_latex(index=False) # Render object to a LaTeX tabular, longtable, or nested table.

In [ ]:
# df = utilities.create_data_frame_counterparts(region_of_interest)
# display(df)
# print(df.to_latex()) # Render object to a LaTeX tabular, longtable, or nested table.

In [ ]:
# print(df[["Source name","RA(deg)","dec.(deg)","Sep.(deg)","Flux points"]].to_latex(index=False)) # Render object to a LaTeX tabular, longtable, or nested table.
# print(df.to_latex(index=False)) # Render object to a LaTeX tabular, longtable, or nested table.

In [ ]:
datasets = Datasets(datasets)
# models=[sky_model_CTA, sky_model]
models=[sky_model_CTA]
dict_leg_style = set_leg_style(dict_leg_style, models = models)

dict_plot_limits = dict(
    energy_bounds = [5e-2, 1e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, models = models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name= sky_model.name)

In [ ]:
# To save datasets and models
datasets.models = [sky_model_CTA]
datasets.write(
    filename=f"datasets_{sky_model_CTA.name}.yaml", filename_models=f"models_{sky_model_CTA.name}.yaml", overwrite=True
)
# # To read only models
# models = Models.read("3fhl_models.yaml")
# print(models)
#To read datasets with models
datasets_HESSJ1826_CTA = Datasets.read(f"datasets_{sky_model_CTA.name}.yaml", filename_models=f"models_{sky_model_CTA.name}.yaml")
print(datasets_HESSJ1826_CTA)

In [ ]:
datasets = Datasets(datasets_HESSJ1826_CTA)
# models=[sky_model_CTA, sky_model]
models=[datasets_HESSJ1826_CTA.models[0]]

dict_plot_limits = dict(
    energy_bounds = [5e-2, 1e3] * u.TeV,
    ylim = [1e-16, 1e-9]
)
plot_SED(dict_plot_limits=dict_plot_limits,datasets=datasets, models = models, dict_leg_style=dict_leg_style, region_of_interest=region_of_interest, name= sky_model.name)